In [3]:
from io import StringIO
import subprocess
import numpy as np
from subprocess import PIPE

class TrentoWrapper:
    """Python wrapper class for executing Trento."""

    def __init__(self, exepath="trento/build/src/trento", **kwargs):
        """
        Initializes a new wrapper class instance.

        Parameters:

            exepath         -- relative or absolute path of the Trento executable
            projectile      -- species of the "A-going" particle; see http://qcd.phy.duke.edu/trento/usage.html
            target          -- species of the "B-going" particle; see ibid.
            sqrts           -- center-of-momentum nucleon-nucleon energy (\sqrt{s_{NN}}), in GeV
            nevents         -- number of events to generate
            norm            -- normalization factor that converts reduced thickness to multiplicity
            nucleon_width   -- Gaussian width of each nucleon, in fm
            num_constit     -- number of subnucleonic degrees of freedom (Gaussian "hotspots") per nucleon
            constit_width   -- Gaussian width of each constituent, in fm
            trento_p        -- reduced thickness parameter for generalized mean
            fluctuation     -- controls gamma distribution used to fluctuate each constituent
            grid_max        -- distance from grid origin to boundary (i.e., half of the width and height), in fm
            grid_step       -- size of a grid cell along either axis, in fm
            random_seed     -- if specified, initializes Trento's pseudorandom number generator to a fixed value for repeatable results
            centrality_bins -- array of low-fraction, high-fraction pairs defining centrality bins (0. is most-central, 1. is most-peripheral)
            event_transform -- function that converts the eight fields Trento outputs for each event into a new representation for binning
            bin_transform   -- function that converts the (transformed) events in each centrality bin into a final representation
        """

        self.exepath = exepath
        self.defaults = kwargs

    def run_explicit(self, projectile="Au", target="Au", sqrts=200., nevents=1000, *,
                     norm=10., nucleon_width=1., num_constits=1, constit_width=None, trento_p=0., fluctuation=1.,
                     grid_max=15., grid_step=0.1, random_seed=None,
                     centrality_bins=[(0.00,0.05), (0.05,0.10), (0.10,0.20), (0.20,0.30), (0.30,0.50),],
                     event_transform=lambda nev, b, npart, mult, e2, e3, e4, e5: (e2, e3),
                     bin_transform=lambda events: np.mean(events, axis=0)
                    ):
        """
        Internal function for executing Trento that declares its arguments explicitly.
        Use `TrentoWrapper.run` instead.
        """

        if (type(projectile) is not str):
            raise ValueError("projectile must be a particle species string; see the Trento documentation for supported values")

        if (type(target) is not str):
            raise ValueError("target must be a particle species string; see the Trento documentation for supported values")

        if not (np.isscalar(sqrts) and np.isfinite(sqrts) and sqrts > 0.):
            raise ValueError("sqrt(s_NN) must be a positive number")

        if not (type(nevents) is int and nevents > 0):
            raise ValueError("number of events must be a positive integer")

        if not (np.isscalar(norm) and np.isfinite(norm) and norm > 0.):
            raise ValueError("norm must be a positive number")

        if not (np.isscalar(nucleon_width) and np.isfinite(nucleon_width) and nucleon_width > 0.):
            raise ValueError("nucleon width must be a positive number")

        if not (type(num_constits) is int and num_constits > 0):
            raise ValueError("number of constituents must be a positive integer")

        if constit_width is None:
            constit_width = nucleon_width

        if not (np.isscalar(constit_width) and np.isfinite(constit_width) and 0. < constit_width <= nucleon_width):
            raise ValueError("constituent width must be a positive number not greater than nucleon width")

        if (num_constits == 1) != (constit_width == nucleon_width):
            raise ValueError("constituent width must match nucleon width if and only if there is a single constituent")
                
        if not (np.isscalar(trento_p) and np.isfinite(trento_p)):
            raise ValueError("reduced thickness parameter 'p' must be a finite number")

        if not (np.isscalar(fluctuation) and np.isfinite(fluctuation) and fluctuation > 0.):
            raise ValueError("fluctuation must be a positive number")

        if not (np.isscalar(grid_max) and np.isfinite(grid_max) and grid_max > 0.):
            raise ValueError("grid maximum must be a positive number")

        if not (np.isscalar(grid_step) and np.isfinite(grid_step) and 0. < grid_step < 2.*grid_max):
            raise ValueError("grid step must be a positive number less than the grid extent")

        if (random_seed is not None) and not (type(random_seed) is int and random_seed > 0):
            raise ValueError("random seed must be a positive integer")

        if centrality_bins is not None:
            if not callable(bin_transform):
                raise ValueError("bin transform must be a function when performing centrality binning")

            centrality_bins = np.array(centrality_bins)

            if not ( centrality_bins.ndim == 2 and len(centrality_bins) > 0 and centrality_bins.shape[1] == 2 and
                     np.issctype(centrality_bins.dtype) and np.all(np.isfinite(centrality_bins)) and
                     np.all(0. <= centrality_bins[:,0]) and np.all(centrality_bins[:,0] < centrality_bins[:,1]) and np.all(centrality_bins[:,1] <= 1.) ):
                raise ValueError("centrality bins must be an array of one or more pairs of intervals in [0,1]")

        cross_section = {
              "62.4": 3.60,     # 1509.06727
             "193.0": 4.2,
             "200.0": 4.23,     # 1509.06727
            "2760.0": 6.4,      # 1108.6027
            "5020.0": 7.0,      # 1210.3615
            "5440.0": 7.1,      # 1904.08290
            "7000.0": 7.32,     # 1208.4968
        }[f"{sqrts:.1f}"]  # convert to string to avoid issues with comparing real numbers

        proc = subprocess.run( tuple(map(str,
                                (
                                    self.exepath,
                                    projectile, target, nevents,
                                    "-x", cross_section,
                                    "-n", norm,
                                    "-p", trento_p,
                                    "-k", fluctuation,
                                    "-w", nucleon_width,
                                    "-m", num_constits,
                                    "-v", constit_width,
                                    "--grid-max", grid_max,
                                    "--grid-step", grid_step,
                                ) + (("--random-seed", random_seed,) if random_seed is not None else ())
                               )),
                               stdout=PIPE, stderr=PIPE,
                               encoding="us-ascii"
                             )

        events = []
        for line in StringIO(proc.stdout):
            fields = line.split()
            if (len(fields) != 8):
                if (len(fields) == 0): continue
                raise ValueError(f"encountered invalid Trento output: '{line}'")

            nev, b, npart, mult, e2, e3, e4, e5 = int(fields[0]), float(fields[1]), int(fields[2]), *map(float, fields[3:])

            if not ( (nev == len(events)) and np.all(np.isfinite((b,mult,e2,e3,e4,e5,))) and (mult > 0.) and min(b,e2,e3,e4,e5) >= 0. ):
                raise ValueError(f"encountered invalid Trento output: '{line}'")

            events.append((mult,) + tuple(event_transform(nev, b, npart, mult, e2, e3, e4, e5) if event_transform else ()))

        if (len(events) != nevents):
            raise ValueError(f"parsed {len(events)} Trento event(s) but expected {nevents}")

        events.sort(reverse=True, key=lambda _: _[0])  # sort in descending order by multiplicity, for centrality selection
        events = np.array(events)

        if centrality_bins is not None:
            binned = []

            for centrange in centrality_bins:
                ilo, ihi = ( int(round(len(events) * cent)) for cent in centrange )
                if (ilo >= ihi):
                    raise ValueError(f"no events in centrality bin {list(centrange)}; try increasing the number of events and/or widening the centrality bin")

                binned.append( bin_transform(events[ilo:ihi]) )

            events = np.array(binned)

        return events

    def run(self, *args, **kwargs):
        """
        Executes Trento with the specified configuration and returns a processed representation of the generated events.

        Positional parameters:

            projectile
            target
            sqrts
            nevents

        See constructor documentation for a list of recognized keyword parameters.
        """

        allargs = dict(self.defaults)
        allargs.update(dict(zip( ("projectile","target","sqrts","nevents"), args )))
        allargs.update(kwargs)
        return self.run_explicit(**allargs)

In [5]:
def Example1():
    """Generates a single Trento event and displays it."""
    trento = TrentoWrapper()
    # specify p-Pb 5.02 TeV, no centrality bins (so that the raw event is captured), and a per-event transformation that returns all the fields (otherwise only mult would be returned)
    event  = trento.run(projectile="p", target="Pb", sqrts=5020., nevents=1, centrality_bins=None, event_transform=lambda *args: args)[0]

    for label, value in zip(("event #", "impact parameter", "num participants", "multiplicity", "epsilon_2", "epsilon_3", "epsilon_4", "epsilon_5"), event[1:]):  # column 0 = mult, which is redundant here because column 4 is also mult, so skip it
        print(f"{label:18s} =   {value}")

#Example1()

event #            =   0.0
impact parameter   =   4.4985717965
num participants   =   11.0
multiplicity       =   13.518736312
epsilon_2          =   0.0117480347
epsilon_3          =   0.0846836837
epsilon_4          =   0.0979993189
epsilon_5          =   0.0568427224


In [6]:
def Example2():
    """Averages many events into two centrality bins."""
    # for convenience, default values can be supplied to the constructor and overwritten later if desired
    trento = TrentoWrapper(projectile="Cu", target="Au", sqrts=62.4, nucleon_width=0.8, num_constits=6, constit_width=0.4)
    binned = trento.run( nevents=1000, centrality_bins=[(0.00,0.05), (0.40,0.60)],
                         event_transform=lambda nev, b, npart, mult, e2, e3, e4, e5: (b, npart, e2, e3),  # retain mult (implicitly), impact parameter, num participants, epsilon_2, and epsilon_3
                         bin_transform=lambda events: (len(events),) + tuple(events.mean(axis=0)) )       # average all quantities but also retain original number of events

    labels = ("n_evts", "<mult>", "<b>", "<N_part>", "<eps_2>", "<eps_3>")
    for caption, row in zip(("Central", "Peripheral"), binned):
        print(f"{caption+':':15s}", ", ".join([ f"{label} = {value:.4g}" for label, value in zip(labels, row) ]))

#Example2()

Central:        n_evts = 50, <mult> = 793.5, <b> = 2.061, <N_part> = 190.4, <eps_2> = 0.1212, <eps_3> = 0.1443
Peripheral:     n_evts = 200, <mult> = 97.78, <b> = 9.518, <N_part> = 30.51, <eps_2> = 0.4165, <eps_3> = 0.2601


In [7]:
def Example3():
    """Performs multiple runs corresponding to points in a very simple "design,"
       averages them in two centrality bins, and computes a derived observable."""

    trento = TrentoWrapper(
        projectile="Au", target="Au", sqrts=200., nevents=1000,
        centrality_bins=[(0.00,0.06), (0.45,0.55)],
        event_transform=lambda nev, b, npart, mult, e2, e3, e4, e5: (e2,),  # intermediate column 1 = e2 (column 0 is always mult)
        bin_transform=lambda events: (events[:,0].mean(), (events[:,1]**4).mean() / (events[:,1]**2).mean()**2)  # final column 0 = mult, column 1 = <e_2^4>/<e_2^2>^2
    )

    design = [
        # norm, p, fluctuation, w
        [ 10.,  0.,   1.,  1.  ],
        [ 11.,  0.67, 1.1, 0.8 ],
        [ 12., -0.67, 0.9, 0.6 ],
    ]

    results = []

    for params in design:
        results.append( trento.run(**dict(zip(("norm", "trento_p", "fluctuation", "nucleon_width"), params))).ravel() )

    print("Des. pt.    Central <mult>      Central ratio       Peripheral <mult>   Peripheral ratio")
    for despt, (centralmult, centralratio, periphmult, periphratio) in enumerate(results):
        print(f"{despt:8d}  {centralmult:18.10g}  {centralratio:18.10g}  {periphmult:18.10g}  {periphratio:18.10g}")

#Example3()

Des. pt.    Central <mult>      Central ratio       Peripheral <mult>   Peripheral ratio
       0         1611.169891          2.03310704         193.2922746         1.310920112
       1         1900.532557         2.098715625          324.121618         1.575610448
       2         1789.720033         2.190879764         231.6371415         1.265212917


In [5]:
def Example3a():
    """
    Same as above, just written differently for clarity.
    (Results will differ from above because Trento is using a different random seed every time.)
    """

    trento = TrentoWrapper()

    design = [
        # norm, p, fluctuation, w
        [ 10.,  0.,   1.,  1.  ],
        [ 11.,  0.67, 1.1, 0.8 ],
        [ 12., -0.67, 0.9, 0.6 ],
    ]

    def my_event_transform(nev, b, npart, mult, e2, e3, e4, e5):  # event transform always receives these eight fields for each Trento event
        return (e2,)  # only keep mult (always present; we don't need to return it) and epsilon_2

    def my_bin_transform(events):  # bin transform receives an array of all events in a centrality bin; columns are decided by event transform, rows are events
        mults, e2s = events.T
        return ( mults.mean(), (e2s**4).mean() / (e2s**2).mean()**2, )

    results = []

    for norm, p, k, w in design:
        binned = trento.run(
            "Au", "Au", 200., nevents=1000,
            norm=norm, trento_p=p, fluctuation=k, nucleon_width=w,  # these are the Trento parameters controlled by the design
            centrality_bins=[(0.00,0.06), (0.45,0.55)],
            event_transform=my_event_transform,
            bin_transform=my_bin_transform)

        results.append(binned.ravel())

    print("Des. pt.    Central <mult>      Central ratio       Peripheral <mult>   Peripheral ratio")
    for despt, (centralmult, centralratio, periphmult, periphratio) in enumerate(results):
        print(f"{despt:8d}  {centralmult:18.10g}  {centralratio:18.10g}  {periphmult:18.10g}  {periphratio:18.10g}")

#Example3a()

Des. pt.    Central <mult>      Central ratio       Peripheral <mult>   Peripheral ratio
       0         1629.091383         1.939447281         198.4337394          1.40929065
       1         1843.499408         2.642788395         248.7026086         1.576481627
       2         1710.994297         1.785572679         194.9219519         1.323847129


In [6]:
def Example4():
    """Computes the mean and standard deviation of multiplicity for various numbers of events."""
    trento = TrentoWrapper(projectile="Pb", target="Pb", sqrts=5020., centrality_bins=[[0.20,0.30]],
                           event_transform=None,  # no event transform: bin transform will receive a single column containing multiplicity
                           bin_transform=lambda events: (len(events), events.mean(), events.std()))

    for nev in (100, 1000, 10000):
        binevts, multmean, multstd = trento.run(nevents=nev)[0]
        print(f"{binevts:6g}  {multmean:10.6g} +/- {multstd:10.6g}")

#Example4()

    10     909.527 +/-    64.9433
   100     730.095 +/-    76.5941
  1000     706.832 +/-    87.0472
